In [1]:
import pandas as pd

## read file
def read_file(year,month,day):
    data = []
    if len(str(month)) == 1:
        month_ = '0'+str(month)
    else:
        month_ = str(month)
    if len(str(day)) == 1:
        day_ = '0'+str(day)
    else:
        day_ = str(day)
    datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_'+str(year)+'.'+str(month_)+'.'+str(day_)+'.csv'
    data = pd.read_csv(datapath)
    data_CN14 = data[data.Series == data['Series'][0]]
    data = data_CN14
    return data

In [2]:
data = read_file(2014,1,30)

In [18]:
BestPrice 	BestQuantity

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity,QuantityDifference_
0,CNG14,1,2014-01-30D04:27:08.581903900,BFC92EA46AE44FA4,1,700000,1,,A,682000,1,1


In [26]:
import time 
data['QuantityDifference_'] = data['QuantityDifference']
data_ask = data[(data.BidOrAsk == 'A')].reset_index(drop=True)
data_bid = data[(data.BidOrAsk == 'B')].reset_index(drop=True)
def insert(order_book_data,data_to_insert,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position:]
    return pd.concat((top,data_to_insert,bottom)).reset_index(drop = True)

start  =  time.time()
order_book_bid = []
order_book_ask = []
timestamp = data.TimeStamp[0]
temp_ask = 0
temp_bid = 0
bid = []
ask = []
timestamp_ = []
x1 = data[(data.BidOrAsk == 'A')].TimeStamp.unique()
x2 = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
y = data[data['TimeStamp'].str.contains(timestamp)]
bid.append(y[(y.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # bid
ask.append(y[(y.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # ask
a = bid[0].sort(['Price'],ascending = [False])
b = ask[0].sort(['Price'],ascending = [True])    
order_book_bid.append(a[a.QuantityDifference != 0].reset_index(drop = True))
order_book_ask.append(b[b.QuantityDifference != 0].reset_index(drop = True))
order_book_bid.append(0)
order_book_ask.append(0)

for i in range(len(y),5000,1):
    print i,temp_bid,temp_ask
    if data.BidOrAsk[i] == 'A':
        time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                      int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                      int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            if order_book_bid[temp_bid].Price[0] >= data[i:i+1].Price.iloc[0] & time_second < 32400:
                for k in range(0,len(order_book_bid[temp_bid])):
                    diff = order_book_bid[temp_bid].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                    if order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff >= 0:
                        order_book_bid[temp_bid].QuantityDifference_[k] = diff
                        data[i:i+1].QuantityDifference_.iloc[0] = 0
                        break
                    elif order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff < 0:
                        order_book_bid[temp_bid].QuantityDifference_[k] = 0
                        data[i:i+1].QuantityDifference_.iloc[0] = -diff
                        pass
                    else:
                        break
            if data.TimeStamp[i] == x1[temp_ask]:
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                if time_second > 32400:
                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error1(Ask & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< (len(order_book_ask[temp_ask])-1):
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:#or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error1(Ask & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'   
                            pass
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error1(Ask & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True' 
                        pass
                else:
                    print 'before open market'
            elif data.TimeStamp[i] != x1[temp_ask]:
                if temp_ask == 0:
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                  
                    if time_second > 32400:
                        if position_ == 0 and len(order_book_ask[0]) > 1:
                            if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                                print 'Some error2(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass
                        elif 0 < position_< len(order_book_ask[temp_ask])-1:
                            if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                                print 'Some error2(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                                break
                            else:
                                print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_ask[temp_ask])-1:   
                            if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                                print 'Some error2(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            else:
                                print 'position and Best Price is True'
                                pass
                        elif position_ == 0 and len(order_book_ask[0]) == 1:
                            print 'position and Best Price is True'
                            pass
                    else:
                        print 'before open market'
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    order_book_ask.append(order_book_ask[temp_ask])
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)#int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                    if time_second > 32400:
                        if position_ == 0:
                            if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                                print 'Some error3(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass
                        elif 0 < position_< len(order_book_ask[temp_ask])-1:
                            if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                                print 'Some error3(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                                break
                            else:
                                print 'position and Best Price is True'  
                                continue
                        elif position_ == len(order_book_ask[temp_ask])-1:    
                            if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                                print 'Some error3(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            else:
                                print 'position and Best Price is True'                        
                                pass
                        elif position_ == 0 and len(order_book_ask[0]) == 1:
                            print 'position and Best Price is True'
                            pass
                    else:
                        print 'before open market'
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    
            if data.TimeStamp[i] == x1[temp_ask]: 
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                if time_second > 32400:
                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error4(Ask & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass       
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error4(Ask & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_ask[temp_ask])-1:       
                        if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error4(Ask & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])   
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass                        
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        pass
                else:
                    print 'before open market'                   
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
            elif data.TimeStamp[i] != x1[temp_ask]:
                order_book_ask.append(order_book_ask[temp_ask])
                order_number_ =  data['OrderNumber'][i : i + 1].iloc[0]
                position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_ask = temp_ask + 1
                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error5(Ask & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error5(Ask & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_ask[temp_ask])-1:

                        if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error5(Ask & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass                       
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        pass      
                else:
                    print 'before open market'
                    
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                    
    elif data.BidOrAsk[i] == 'B':
        time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                      int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                      int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
        if int(data[['QuantityDifference']][i:i+1].values) > 0: 
            if order_book_ask[temp_ask].Price[0] <= data[i:i+1].Price.iloc[0] & time_second < 32400:
                for k in range(0,len(order_book_ask[temp_ask])):
                    diff = order_book_ask[temp_ask].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                    if order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff >= 0:
                        order_book_ask[temp_ask].QuantityDifference_[k] = diff
                        data[i:i+1].QuantityDifference_.iloc[0] = 0
                        break
                    elif order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff < 0:
                        order_book_ask[temp_ask].QuantityDifference_[k] = 0
                        data[i:i+1].QuantityDifference_.iloc[0] = -diff
                        pass
                    else:
                        break
            if data.TimeStamp[i] == x2[temp_bid] :               
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                best_price = order_book_bid[temp_bid].Price[0]
                best_quantity = sum(order_book_bid[temp_bid][order_book_bid[temp_bid].Price == best_price].QuantityDifference)             
                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or \
                            best_price != data['BestPrice'][i:i+1].iloc[0] or best_quantity != data['BestQuantity'][i:i+1].iloc[0]:
                            print 'Some error1(Bid & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass     
                    elif 0 < position_< len(order_book_bid[temp_bid])-1: 
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or \
                            best_price != data['BestPrice'][i:i+1].iloc[0] or best_quantity != data['BestQuantity'][i:i+1].iloc[0]: 
                            print 'Some error1(Bid & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'   
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1 and len(order_book_bid[0]) > 1:      
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or \
                            best_price != data['BestPrice'][i:i+1].iloc[0] or best_quantity != data['BestQuantity'][i:i+1].iloc[0]: 
                            print 'Some error1(Bid & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        pass 
                else:
                    print 'before open market'
                    pass
            elif data.TimeStamp[i] != x2[temp_bid]:
                if temp_bid == 0:
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)#int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    best_price = order_book_bid[temp_bid].Price[0]
                    best_quantity = sum(order_book_bid[temp_bid][order_book_bid[temp_bid].Price == best_price].QuantityDifference)            
                    if time_second > 32400:
                        if position_ == 0  and len(order_book_bid[0]) > 1:
                            if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i] or \
                            best_price != data['BestPrice'][i:i+1].iloc[0] or best_quantity != data['BestQuantity'][i:i+1].iloc[0]: 
                                print 'Some error2(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass       
                        elif 0 < position_< len(order_book_bid[temp_bid])-1:
                            if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or \
                                best_price != data['BestPrice'][i:i+1].iloc[0] or best_quantity != data['BestQuantity'][i:i+1].iloc[0]: 
                                print 'Some error2(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_bid[temp_bid])-1:    
                            if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i] or \
                            best_price != data['BestPrice'][i:i+1].iloc[0] or best_quantity != data['BestQuantity'][i:i+1].iloc[0]: 
                                print 'Some error2(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])                                        
                                pass
                            else:
                                print 'position and Best Price is True'
                                pass
                        elif position_ == 0 and len(order_book_bid[0]) == 1:
                                print 'position and Best Price is True'
                                pass
                    else:
                        print 'before open market'
                        pass
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    order_book_bid.append(order_book_bid[temp_bid])
                    temp_bid = temp_bid + 1 
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)   
                    best_price = order_book_bid[temp_bid].Price[0]
                    best_quantity = sum(order_book_bid[temp_bid][order_book_bid[temp_bid].Price == best_price].QuantityDifference) 
                    if time_second > 32400:
                        if position_ == 0 and len(order_book_bid[0]) > 1:
                            if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or \
                                order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i] or \
                                best_price != data['BestPrice'][i:i+1].iloc[0] or best_quantity != data['BestQuantity'][i:i+1].iloc[0]: 
                                print 'Some error3(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass
                        elif 0 < position_< len(order_book_bid[temp_bid])-1:
                            if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or \
                                order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or \
                            best_price != data['BestPrice'][i:i+1].iloc[0] or best_quantity != data['BestQuantity'][i:i+1].iloc[0]: 
                                print 'Some error3(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_bid[temp_bid])-1:
                            if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or \
                                order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i] or \
                                best_price != data['BestPrice'][i:i+1].iloc[0] or best_quantity != data['BestQuantity'][i:i+1].iloc[0]: 
                                print 'Some error3(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            else:
                                print 'position and Best Price is True'                        
                                pass
                        elif position_ == 0 and len(order_book_bid[0]) == 1:
                            print 'position and Best Price is True'
                            pass
                    else:
                        print 'before open market'
                        pass
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    
            if data.TimeStamp[i] == x2[temp_bid]: 
                order_number_ =  data['OrderNumber'][i : i + 1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                best_price = order_book_bid[temp_bid].Price[0]
                best_quantity = sum(order_book_bid[temp_bid][order_book_bid[temp_bid].Price == best_price].QuantityDifference) 
                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or \
                            best_price != data['BestPrice'][i:i+1].iloc[0] or best_quantity != data['BestQuantity'][i:i+1].iloc[0]: 
                            print 'Some error4(Bid & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass               
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or \
                            order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or \
                            best_price != data['BestPrice'][i:i+1].iloc[0] or best_quantity != data['BestQuantity'][i:i+1].iloc[0]:
                            print 'Some error4(Bid & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or \
                            best_price != data['BestPrice'][i:i+1].iloc[0] or best_quantity != data['BestQuantity'][i:i+1].iloc[0]:
                            print 'Some error4(Bid & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        pass
                else:
                    print 'before open market'
                    pass
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]                
            elif data.TimeStamp[i] != x2[temp_bid]:
                order_book_bid.append(order_book_bid[temp_bid])
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_bid = temp_bid + 1
                best_price = order_book_bid[temp_bid].Price[0]
                best_quantity = sum(order_book_bid[temp_bid][order_book_bid[temp_bid].Price == best_price].QuantityDifference) 

                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or \
                            best_price != data['BestPrice'][i:i+1].iloc[0] or best_quantity != data['BestQuantity'][i:i+1].iloc[0]:  
                            print 'Some error5(Bid & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or \
                            order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or \
                            best_price != data['BestPrice'][i:i+1].iloc[0] or best_quantity != data['BestQuantity'][i:i+1].iloc[0]:
                            print 'Some error5(Bid & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1: 
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or \
                            best_price != data['BestPrice'][i:i+1].iloc[0] or best_quantity != data['BestQuantity'][i:i+1].iloc[0]: 
                            print 'Some error5(Bid & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass
                        else:
                            print 'position and Best Price is True'                        
                            pass    
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        pass  
                else:
                    print 'before open market'
                    pass
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
end = time.time()            
print "Total time = %f"%(end - start)

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


13 0 0
before open market
14 1 0
before open market
15 2 0
before open market
16 3 0
before open market
17 4 0
before open market
18 5 0
before open market
19 6 0
before open market
20 7 0
before open market
21 7 1
before open market
22 8 1
before open market
23 9 1
before open market
24 10 1
before open market
25 11 1
before open market
26 11 1
before open market
27 12 1
before open market
28 13 1
before open market
29 14 1
before open market
30 14 2
before open market
31 15 2
before open market
32 15 2
before open market
33 15 3
before open market
34 15 3
before open market
35 15 4
before open market
36 16 4
before open market
37 16 5
before open market
38 16 6
before open market
39 16 6
before open market
40 16 7
before open market
41 16 7
before open market
42 16 8
before open market
43 16 9
before open market
44 16 10
before open market
45 17 10
before open market
46 17 11
before open market
47 17 12
before open market
48 18 12
before open market
49 18 13
before open market
50 18 

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:41: SettingWithCopyWarning: 
A value i

before open market
84 35 30
before open market
85 36 30
before open market
86 37 30
before open market
87 37 31
before open market
88 37 32
before open market
89 38 32
before open market
90 38 33
before open market
91 39 33
before open market
92 39 34
before open market
93 40 34
before open market
94 40 35
before open market
95 41 35
before open market
96 42 35
before open market
97 43 35
before open market
98 44 35
before open market
99 44 36
before open market
100 44 37
before open market
101 44 38
before open market
102 45 38
before open market
103 45 39
before open market
104 45 40
before open market
105 46 40
before open market
106 46 41
before open market
107 46 42
before open market
108 47 42
before open market
109 47 42
before open market
110 47 42
before open market
111 47 42
before open market
112 47 42
before open market
113 47 42
before open market
114 47 42
before open market
115 47 42
before open market
116 47 42
before open market
117 47 42
before open market
118 47 42
b

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:225: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


before open market
261 77 87
before open market
262 77 88
before open market
263 77 89
before open market
264 77 89
before open market
265 77 90
before open market
266 77 91
before open market
267 77 92
before open market
268 77 92
before open market
269 77 93
before open market
270 77 94
before open market
271 77 95
before open market
272 77 95
before open market
273 77 96
before open market
274 77 97
before open market
275 77 98
before open market
276 77 98
before open market
277 77 99
before open market
278 77 100
before open market
279 77 101
before open market
280 77 101
before open market
281 77 102
before open market
282 77 103
before open market
283 77 104
before open market
284 77 104
before open market
285 77 105
before open market
286 78 105
before open market
287 78 105
before open market
288 78 105
before open market
289 78 105
before open market
290 78 105
before open market
291 78 105
before open market
292 78 105
before open market
293 78 105
before open market
294 78 1

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:183: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


803 119 186
before open market
804 119 186
before open market
805 119 186
before open market
806 119 186
before open market
807 119 186
before open market
808 119 186
before open market
809 119 186
before open market
810 119 186
before open market
811 119 187
before open market
812 119 188
before open market
813 120 188
before open market
814 120 188
before open market
815 120 188
before open market
816 120 189
before open market
817 121 189
before open market
818 121 190
before open market
819 121 191
before open market
820 121 192
before open market
821 121 193
before open market
822 121 193
before open market
823 121 194
before open market


/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


824 121 195
before open market
825 121 196
before open market
826 121 196
before open market
827 121 197
before open market
828 122 197
before open market
829 123 197
before open market
830 124 197
position and Best Price is True
831 124 198
position and Best Price is True
832 124 199
position and Best Price is True
833 124 199
position and Best Price is True
834 124 199
position and Best Price is True
835 124 199
position and Best Price is True
836 124 200
position and Best Price is True
837 124 200
position and Best Price is True
838 124 201
position and Best Price is True
839 124 202
position and Best Price is True
840 124 203
position and Best Price is True
841 124 204
position and Best Price is True
842 124 205
position and Best Price is True
843 124 206
position and Best Price is True
844 124 207
position and Best Price is True
845 124 208
position and Best Price is True
846 124 209
position and Best Price is True
847 125 209
position and Best Price is True
848 126 209
position a

In [31]:
data[865:868]

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity,QuantityDifference_
865,CNG14,866,2014-01-30D09:00:03.508037600,015A453E967E789E,28,667000,1,,B,672000,46,1
866,CNG14,867,2014-01-30D09:00:03.531037600,2BA4612B6D044F44,29,667000,1,,B,672000,46,1
867,CNG14,868,2014-01-30D09:00:03.547037600,A3A4E12BAF8091C0,21,668000,-1,,B,672000,47,-1


In [30]:
temp_ask,temp_bid,best_price,best_quantity

(209, 143, 672000, 46)

In [28]:
order_book_bid[143]

,Price,OrderNumber,QuantityDifference,QuantityDifference_
0,672000,AD19452667B04C70,15,15
1,672000,AD91452E50DD359D,15,15
2,672000,AB2C6123BF3DA17D,1,1
3,672000,AD19452E2CD01190,15,15
4,671500,F00DAF85379C189C,10,10
5,671000,7CA2E30920EE052E,48,50
6,671000,D4A2621D51303010,20,20
7,670500,74A2E7010AE6EF27,22,22
8,670500,A158C53E826F674F,30,30
9,670000,7071660D57A33AC3,1,1


In [5]:
order_book_ask[1561]

,Price,OrderNumber,QuantityDifference,QuantityDifference_
0,673000,2364ED2BFD07DF47,39,50
1,673000,2546C56662A847A8,1,1
2,673000,703B6205650344C3,10,10
3,673500,7021EE45C802AB22,2,2
4,673500,A320652B37571997,1,1
5,673500,562D86D190FE717E,10,10
6,673500,583B66051CD2FC93,10,10
7,673500,5424E24D67034663,10,10
8,673500,54ABE205282E0B6E,10,10
9,673500,0B584D3E5A2E3F0E,45,45


In [6]:
position_

0

In [156]:
data[108:109]

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity,QuantityDifference_
108,CNG14,109,2014-01-30D08:50:29.121495300,FA6ACA4D2D540C54,1,674500,1,,B,674500,10,1


In [8]:
#order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] #> data['Price'][i:i+1].iloc[0]

In [9]:
#data['Price'][i:i+1].iloc[0]

In [147]:
import time 
data['QuantityDifference_'] = data['QuantityDifference']
data_ask = data[(data.BidOrAsk == 'A')].reset_index(drop=True)
data_bid = data[(data.BidOrAsk == 'B')].reset_index(drop=True)
def insert(order_book_data,data_to_insert,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position:]
    return pd.concat((top,data_to_insert,bottom)).reset_index(drop = True)

start  =  time.time()
order_book_bid = []
order_book_ask = []
timestamp = data.TimeStamp[0]
temp_ask = 0
temp_bid = 0
bid = []
ask = []
timestamp_ = []
x1 = data[(data.BidOrAsk == 'A')].TimeStamp.unique()
x2 = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
y = data[data['TimeStamp'].str.contains(timestamp)]
bid.append(y[(y.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # bid
ask.append(y[(y.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # ask
a = bid[0].sort(['Price'],ascending = [False])
b = ask[0].sort(['Price'],ascending = [True])    
order_book_bid.append(a[a.QuantityDifference != 0].reset_index(drop = True))
order_book_ask.append(b[b.QuantityDifference != 0].reset_index(drop = True))
order_book_bid.append(0)
order_book_ask.append(0)

for i in range(len(y),1000,1):
    print i,temp_bid,temp_ask
    if data.BidOrAsk[i] == 'A':
        if order_book_bid[temp_bid].Price[0] >= data[i:i+1].Price.iloc[0]:
            for k in range(0,len(order_book_bid[temp_bid])):
                diff = order_book_bid[temp_bid].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                print 'diff_ask = %d'%(diff)
                if order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff >= 0:
                    order_book_bid[temp_bid].QuantityDifference_[k] = diff
                    data[i:i+1].QuantityDifference_.iloc[0] = 0
                    print 'hello1 + %d'%(k)
                    break
                elif order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff < 0:
                    order_book_bid[temp_bid].QuantityDifference_[k] = 0
                    data[i:i+1].QuantityDifference_.iloc[0] = -diff
                    print 'hello2 + %d'%(k)
                    pass
                else:
                    break
        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            if data.TimeStamp[i] == x1[temp_ask]:
                #time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                #              int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                #              int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
                #if 31500 <= time_second < 31800 :
                #    length_order_book_zero = len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price <= data['Price'][i:i+1].iloc[0]) & (order_book_ask[temp_ask].QuantityDifference_ == 0)])
                #else:
                #    length_order_book_zero = 0
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 #+ length_order_book_zero
                order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                if position_ == 0 and len(order_book_ask[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Ask & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass
                elif 0 < position_< (len(order_book_ask[temp_ask])-1):
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:#or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Ask & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'   
                        pass
                elif position_ == len(order_book_ask[temp_ask])-1:
                    if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Ask & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                
                        pass
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True' 
                    pass
                
            elif data.TimeStamp[i] != x1[temp_ask]:
                if temp_ask == 0:
                    #time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                    #              int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                    #              int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
                    #if 31500 < time_second < 31800 :
                    #    length_order_book_zero = len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price <= data['Price'][i:i+1].iloc[0]) & (order_book_ask[temp_ask].QuantityDifference_ == 0)])
                    #else:
                    #    length_order_book_zero = 0
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 #+ length_order_book_zero
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                  
                   
                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_ask[temp_ask])-1:   
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        else:
                            print 'position and Best Price is True'
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        pass
                else:
                    #time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                    #              int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                    #              int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
                    #if 31500 < time_second < 31800 :
                    #    length_order_book_zero = len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price <= data['Price'][i:i+1].iloc[0]) & (order_book_ask[temp_ask].QuantityDifference_ == 0)])
                    #else:
                    #    length_order_book_zero = 0
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 #+ length_order_book_zero
                    order_book_ask.append(order_book_ask[temp_ask])
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)#int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                    if position_ == 0:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error3(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error3(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:    
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error3(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        pass
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x1[temp_ask]: 
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                if position_ == 0 and len(order_book_ask[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error4(Ask & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass       
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error4(Ask & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'  
                        pass
                elif position_ == len(order_book_ask[temp_ask])-1:       
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error4(Ask & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])   
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        pass                        
                    else:
                        print 'position and Best Price is True'                        
                        pass
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True'
                    pass
    
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]

            elif data.TimeStamp[i] != x1[temp_ask]:
                order_book_ask.append(order_book_ask[temp_ask])
                order_number_ =  data['OrderNumber'][i : i + 1].iloc[0]
                position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_ask = temp_ask + 1

                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error5(Ask & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error5(Ask & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                        break
                    else:
                        print 'position and Best Price is True'  
                        pass
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error5(Ask & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        pass                       
                    else:
                        print 'position and Best Price is True'                        
                        pass
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True'
                    pass      
                    
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                    
    elif data.BidOrAsk[i] == 'B':
        if order_book_ask[temp_ask].Price[0] <= data[i:i+1].Price.iloc[0]:
            for k in range(0,len(order_book_ask[temp_ask])):
                diff = order_book_ask[temp_ask].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                print 'diff_bid = %d'%(diff)
                if order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff >= 0:
                    order_book_ask[temp_ask].QuantityDifference_[k] = diff
                    data[i:i+1].QuantityDifference_.iloc[0] = 0
                    print 'hello3 + %d'%(k)
                    break
                elif order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff < 0:
                    order_book_ask[temp_ask].QuantityDifference_[k] = 0
                    data[i:i+1].QuantityDifference_.iloc[0] = -diff
                    print 'hello4 + %d'%(k)
                    pass
                else:
                    break
        if int(data[['QuantityDifference']][i:i+1].values) > 0: 
            if data.TimeStamp[i] == x2[temp_bid] :               
                #time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                #              int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                #              int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
                #if 31500 <= time_second < 31800 :
                #    length_order_book_zero = len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price > data['Price'][i:i+1].iloc[0]) & (order_book_bid[temp_bid].QuantityDifference_ == 0)])
                #else:
                #    length_order_book_zero = 0
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 #+ length_order_book_zero
                order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Bid & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass     
                elif 0 < position_< len(order_book_bid[temp_bid])-1: 
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Bid & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'   
                        pass
                elif position_ == len(order_book_bid[temp_bid])-1 and len(order_book_bid[0]) > 1:      
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Bid & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                
                        pass
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    pass  
            elif data.TimeStamp[i] != x2[temp_bid]:
                if temp_bid == 0:
                    #time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                    #              int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                    #              int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
                    #if 31500 <= time_second < 31800 :
                    #    length_order_book_zero = len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price > data['Price'][i:i+1].iloc[0]) & (order_book_bid[temp_bid].QuantityDifference_ == 0)])
                    #else:
                    #    length_order_book_zero = 0
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 #+ length_order_book_zero
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)#int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    if position_ == 0  and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass       
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1:    
                        if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])                                        
                            pass
                        else:
                            print 'position and Best Price is True'
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                            print 'position and Best Price is True'
                            pass
                else:
                    #time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                    #              int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                    #              int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
                    #if 31500 <= time_second < 31800 :
                    #    length_order_book_zero = len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price > data['Price'][i:i+1].iloc[0]) & (order_book_bid[temp_bid].QuantityDifference_ == 0)])
                    #else:
                    #    length_order_book_zero = 0                    
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 #+ length_order_book_zero
                    order_book_bid.append(order_book_bid[temp_bid])
                    temp_bid = temp_bid + 1 
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)   
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error3(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error3(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error3(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        pass
                        
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x2[temp_bid]: 
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error4(Bid & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass               
                elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error4(Bid & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'  
                        pass
                elif position_ == len(order_book_bid[temp_bid])-1:
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                        print 'Some error4(Bid & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        pass
                    else:
                        #print 'position and Best Price is True'                        
                        pass
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    pass
                    
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]                
            elif data.TimeStamp[i] != x2[temp_bid]:
                order_book_bid.append(order_book_bid[temp_bid])
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_bid = temp_bid + 1
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error5(Bid & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass
                elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error5(Bid & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        pass
                elif position_ == len(order_book_bid[temp_bid])-1: 
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error5(Bid & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        pass
                    else:
                        print 'position and Best Price is True'                        
                        pass    
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    pass  
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
end = time.time()            
print "Total time = %f"%(end - start)

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


13 0 0
position and Best Price is True
14 1 0
position and Best Price is True
15 2 0
position and Best Price is True
16 3 0
position and Best Price is True
17 4 0
position and Best Price is True
18 5 0
position and Best Price is True
19 6 0
position and Best Price is True
20 7 0
position and Best Price is True
21 7 1
position and Best Price is True
22 8 1
position and Best Price is True
23 9 1
position and Best Price is True
24 10 1
position and Best Price is True
25 11 1
26 11 1
position and Best Price is True
27 12 1
position and Best Price is True
28 13 1
position and Best Price is True
29 14 1
position and Best Price is True
30 14 2
position and Best Price is True
31 15 2
position and Best Price is True
32 15 2
position and Best Price is True
33 15 3
position and Best Price is True
34 15 3
position and Best Price is True
35 15 4
position and Best Price is True
36 16 4
position and Best Price is True
37 16 5
position and Best Price is True
38 16 6
position and Best Price is True
39 

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:247: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:248: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


hello3 + 0
position and Best Price is True
73 25 29
diff_bid = 16
hello3 + 0
position and Best Price is True
74 26 29
diff_bid = 15
hello3 + 0
position and Best Price is True
75 27 29
diff_bid = 17
hello3 + 0
position and Best Price is True
76 28 29
diff_bid = 18
hello3 + 0
position and Best Price is True
77 29 29
diff_bid = 17
hello3 + 0
position and Best Price is True
78 30 29
diff_bid = 16
hello3 + 0
position and Best Price is True
79 31 29
diff_bid = 15
hello3 + 0
position and Best Price is True
80 32 29
diff_bid = 14
hello3 + 0
position and Best Price is True
81 33 29
diff_bid = 13
hello3 + 0
position and Best Price is True
82 34 29
diff_bid = 12
hello3 + 0
position and Best Price is True
83 35 29
diff_ask = -3
hello2 + 0
diff_ask = -3


/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


hello2 + 1
diff_ask = -3
hello2 + 2
diff_ask = -3
hello2 + 3
diff_ask = -3
hello2 + 4
diff_ask = -3
hello2 + 5
diff_ask = -3
hello2 + 6
diff_ask = -3
hello2 + 7
diff_ask = 47
position and Best Price is True
84 35 30
position and Best Price is True
85 36 30
diff_bid = 2
hello3 + 0
position and Best Price is True
86 37 30
position and Best Price is True
87 37 31
diff_ask = 3


/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


hello1 + 0
position and Best Price is True
88 37 32
diff_bid = 11
hello3 + 0
position and Best Price is True
89 38 32
diff_ask = 0
hello1 + 0
position and Best Price is True
90 38 33
position and Best Price is True
91 39 33
position and Best Price is True
92 39 34
position and Best Price is True
93 40 34
diff_ask = 3
hello1 + 0
position and Best Price is True
94 40 35
position and Best Price is True
95 41 35
position and Best Price is True
96 42 35
position and Best Price is True
97 43 35
position and Best Price is True
98 44 35
position and Best Price is True
99 44 36
diff_ask = 2
hello1 + 0
position and Best Price is True
100 44 37
diff_ask = 0
hello1 + 0
position and Best Price is True
101 44 38
position and Best Price is True
102 45 38
diff_ask = -6
hello2 + 0
diff_ask = -6
hello2 + 1
diff_ask = -6
hello2 + 2
diff_ask = -6
hello2 + 3
diff_ask = -6
hello2 + 4
diff_ask = -6
hello2 + 5
diff_ask = -6
hello2 + 6
diff_ask = -6
hello2 + 7
diff_ask = -6
hello2 + 8
diff_ask = -6
hello2 + 9


In [10]:
len(order_book_bid),len(order_book_ask)

(358, 286)

In [13]:
#order_book_bid[357]

In [15]:
#order_book_ask[285]

In [16]:
len(data[0:len(data)][(data[0:len(data)].BidOrAsk == 'A')].TimeStamp.unique()),len(data[0:len(data)][(data[0:len(data)].BidOrAsk == 'B')].TimeStamp.unique())

(18184, 18315)

In [26]:
import time
start = time.time()
order_book_bid_merge = []
order_book_ask_merge = []
for i in range(0,len(order_book_bid)):
    a = order_book_bid[i][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()#.sort(['Price'],ascending = [False])
    order_book_bid_merge.append(pd.DataFrame(a.values))
for i in range(0,len(order_book_ask)):
    a = order_book_ask[i][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()#.sort(['Price'],ascending = [True])
    order_book_ask_merge.append(pd.DataFrame(a.values))
end = time.time()            
print ("Total time = %f"%(end - start)) 


Total time = 226.436287


In [12]:
import time 
data['QuantityDifference_'] = data['QuantityDifference']
data_ask = data[(data.BidOrAsk == 'A')].reset_index(drop=True)
data_bid = data[(data.BidOrAsk == 'B')].reset_index(drop=True)
def insert(order_book_data,data_to_insert,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position:]
    return pd.concat((top,data_to_insert,bottom)).reset_index(drop = True)

start  =  time.time()
order_book_bid = []
order_book_ask = []
timestamp = data.TimeStamp[0]
temp_ask = 0
temp_bid = 0
bid = []
ask = []
timestamp_ = []
x1 = data[(data.BidOrAsk == 'A')].TimeStamp.unique()
x2 = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
y = data[data['TimeStamp'].str.contains(timestamp)]
bid.append(y[(y.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # bid
ask.append(y[(y.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # ask
a = bid[0].sort(['Price'],ascending = [False])
b = ask[0].sort(['Price'],ascending = [True])    
order_book_bid.append(a[a.QuantityDifference != 0].reset_index(drop = True))
order_book_ask.append(b[b.QuantityDifference != 0].reset_index(drop = True))
order_book_bid.append(0)
order_book_ask.append(0)

for i in range(len(y),1000,1):
    print i,temp_bid,temp_ask
    if data.BidOrAsk[i] == 'A':
        if order_book_bid[temp_bid].Price[0] >= data[i:i+1].Price.iloc[0]:
            for k in range(0,len(order_book_bid[temp_bid])):
                diff = order_book_bid[temp_bid].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                print 'diff_ask = %d'%(diff)
                if order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff >= 0:
                    order_book_bid[temp_bid].QuantityDifference_[k] = diff
                    data[i:i+1].QuantityDifference_.iloc[0] = 0
                    print 'hello1 + %d'%(k)
                    break
                elif order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff < 0:
                    order_book_bid[temp_bid].QuantityDifference_[k] = 0
                    data[i:i+1].QuantityDifference_.iloc[0] = -diff
                    print 'hello2 + %d'%(k)
                    pass
                else:
                    break
        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            if data.TimeStamp[i] == x1[temp_ask]:
                time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                              int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                              int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
                print 'time second = %d'%(time_second)
                #if 31800 + 29 == time_second:#31800 + 29 <= time_second < 32400 :
                #    length_order_book_zero = len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price <= data['Price'][i:i+1].iloc[0]) & (order_book_ask[temp_ask].QuantityDifference_ == 0)])
                #else:
                #    length_order_book_zero = 0
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 #+ length_order_book_zero
                order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                if position_ == 0 and len(order_book_ask[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Ask & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass
                elif 0 < position_< (len(order_book_ask[temp_ask])-1):
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:#or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Ask & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'   
                        pass
                elif position_ == len(order_book_ask[temp_ask])-1:
                    if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Ask & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                
                        pass
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True' 
                    pass
                
            elif data.TimeStamp[i] != x1[temp_ask]:
                if temp_ask == 0:
                    time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                                  int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                                  int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
                    print 'time second = %d'%(time_second)
                    #if 31800 + 29 == time_second:#31800 + 29 <= time_second < 32400 :
                    #    length_order_book_zero = len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price <= data['Price'][i:i+1].iloc[0]) & (order_book_ask[temp_ask].QuantityDifference_ == 0)])
                    #else:
                    #    length_order_book_zero = 0
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 #+ length_order_book_zero
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                  
                   
                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_ask[temp_ask])-1:   
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        else:
                            print 'position and Best Price is True'
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        pass
                else:
                    time_second == int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                                  int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                                  int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
                    print 'time second = %d'%(time_second)
                    #if 31800 + 29 == time_second:#31800 + 29 <= time_second < 32400 :
                    #    length_order_book_zero = len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price <= data['Price'][i:i+1].iloc[0]) & (order_book_ask[temp_ask].QuantityDifference_ == 0)])
                    #else:
                    #    length_order_book_zero = 0
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 #+ length_order_book_zero
                    order_book_ask.append(order_book_ask[temp_ask])
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)#int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                    if position_ == 0:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error3(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error3(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:    
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error3(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        pass
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x1[temp_ask]: 
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                if position_ == 0 and len(order_book_ask[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error4(Ask & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass       
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error4(Ask & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'  
                        pass
                elif position_ == len(order_book_ask[temp_ask])-1:       
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error4(Ask & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])   
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        pass                        
                    else:
                        print 'position and Best Price is True'                        
                        pass
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True'
                    pass
    
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]

            elif data.TimeStamp[i] != x1[temp_ask]:
                order_book_ask.append(order_book_ask[temp_ask])
                order_number_ =  data['OrderNumber'][i : i + 1].iloc[0]
                position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_ask = temp_ask + 1

                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error5(Ask & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error5(Ask & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                        break
                    else:
                        print 'position and Best Price is True'  
                        pass
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error5(Ask & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        pass                       
                    else:
                        print 'position and Best Price is True'                        
                        pass
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True'
                    pass      
                    
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                    
    elif data.BidOrAsk[i] == 'B':
        if order_book_ask[temp_ask].Price[0] <= data[i:i+1].Price.iloc[0]:
            for k in range(0,len(order_book_ask[temp_ask])):
                diff = order_book_ask[temp_ask].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                print 'diff_bid = %d'%(diff)
                if order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff >= 0:
                    order_book_ask[temp_ask].QuantityDifference_[k] = diff
                    data[i:i+1].QuantityDifference_.iloc[0] = 0
                    print 'hello3 + %d'%(k)
                    break
                elif order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff < 0:
                    order_book_ask[temp_ask].QuantityDifference_[k] = 0
                    data[i:i+1].QuantityDifference_.iloc[0] = -diff
                    print 'hello4 + %d'%(k)
                    pass
                else:
                    break
        if int(data[['QuantityDifference']][i:i+1].values) > 0: 
            if data.TimeStamp[i] == x2[temp_bid] :               
                time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                              int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                              int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
                print 'time second = %d'%(time_second)
                if 31800 + 29 == time_second or 31942 == time_second:#31800 + 29 <= time_second < 32400 :
                    length_order_book_zero = len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price >= data['Price'][i:i+1].iloc[0]) & (order_book_bid[temp_bid].QuantityDifference_ == 0)])
                else:
                    length_order_book_zero = 0
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 + length_order_book_zero
                order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Bid & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass     
                elif 0 < position_< len(order_book_bid[temp_bid])-1: 
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Bid & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'   
                        pass
                elif position_ == len(order_book_bid[temp_bid])-1 and len(order_book_bid[0]) > 1:      
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Bid & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                
                        pass
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    pass  
            elif data.TimeStamp[i] != x2[temp_bid]:
                if temp_bid == 0:
                    time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                                  int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                                  int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
                    print 'time second = %d'%(time_second)
                    if 31800 + 29 == time_second or 31942 == time_second:#31800 + 29 <= time_second < 32400 :
                        length_order_book_zero = len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price >= data['Price'][i:i+1].iloc[0]) & (order_book_bid[temp_bid].QuantityDifference_ == 0)])
                    else:
                        length_order_book_zero = 0
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 + length_order_book_zero
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)#int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    if position_ == 0  and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass       
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1:    
                        if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])                                        
                            pass
                        else:
                            print 'position and Best Price is True'
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                            print 'position and Best Price is True'
                            pass
                else:
                    time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                                  int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                                  int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
                    print 'time second = %d'%(time_second)
                    if 31800 + 29 == time_second or 31942 == time_second:#31800 + 29 <= time_second < 32400 :
                        length_order_book_zero = len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price >= data['Price'][i:i+1].iloc[0]) & (order_book_bid[temp_bid].QuantityDifference_ == 0)])
                    else:
                        length_order_book_zero = 0                    
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 + length_order_book_zero
                    order_book_bid.append(order_book_bid[temp_bid])
                    temp_bid = temp_bid + 1 
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)   
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error3(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error3(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error3(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        pass
                        
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x2[temp_bid]: 
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error4(Bid & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass               
                elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error4(Bid & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'  
                        pass
                elif position_ == len(order_book_bid[temp_bid])-1:
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                        print 'Some error4(Bid & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        pass
                    else:
                        #print 'position and Best Price is True'                        
                        pass
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    pass
                    
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]                
            elif data.TimeStamp[i] != x2[temp_bid]:
                order_book_bid.append(order_book_bid[temp_bid])
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_bid = temp_bid + 1
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error5(Bid & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass
                elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error5(Bid & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        pass
                elif position_ == len(order_book_bid[temp_bid])-1: 
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error5(Bid & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        pass
                    else:
                        print 'position and Best Price is True'                        
                        pass    
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    pass  
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
end = time.time()            
print "Total time = %f"%(end - start)

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


13 0 0
time second = 31500
position and Best Price is True
14 1 0
time second = 31501
position and Best Price is True
15 2 0
time second = 31501
position and Best Price is True
16 3 0
time second = 31501
position and Best Price is True
17 4 0
time second = 31501
position and Best Price is True
18 5 0
time second = 31501
position and Best Price is True
19 6 0
position and Best Price is True
20 7 0
time second = 31526
position and Best Price is True
21 7 1
time second = 31530
position and Best Price is True
22 8 1
time second = 31531
position and Best Price is True
23 9 1
time second = 31544
position and Best Price is True
24 10 1
position and Best Price is True
25 11 1
26 11 1
position and Best Price is True
27 12 1
time second = 31553
position and Best Price is True
28 13 1
time second = 31553
position and Best Price is True
29 14 1
time second = 31553
position and Best Price is True
30 14 2
time second = 31560
position and Best Price is True
31 15 2
time second = 31560
position and Be

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


hello3 + 0
time second = 31575
position and Best Price is True
67 23 25
time second = 31575
position and Best Price is True
68 23 26
time second = 31575
position and Best Price is True
69 23 27
time second = 31575
position and Best Price is True
70 23 28
time second = 31586
position and Best Price is True
71 24 28
time second = 31586
position and Best Price is True
72 24 29
diff_bid = 18
hello3 + 0
time second = 31599
position and Best Price is True
73 25 29
diff_bid = 16
hello3 + 0
time second = 31604
position and Best Price is True
74 26 29
diff_bid = 15
hello3 + 0
time second = 31606
position and Best Price is True
75 27 29
diff_bid = 17
hello3 + 0
position and Best Price is True
76 28 29
diff_bid = 18
hello3 + 0
position and Best Price is True
77 29 29
diff_bid = 17
hello3 + 0
time second = 31615
position and Best Price is True
78 30 29
diff_bid = 16
hello3 + 0
time second = 31626
position and Best Price is True
79 31 29
diff_bid = 15
hello3 + 0
time second = 31634
position and Bes

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


hello2 + 2
diff_ask = -3
hello2 + 3
diff_ask = -3
hello2 + 4
diff_ask = -3
hello2 + 5
diff_ask = -3
hello2 + 6
diff_ask = -3
hello2 + 7
diff_ask = 47
time second = 31653
position and Best Price is True
84 35 30
time second = 31655
position and Best Price is True
85 36 30
diff_bid = 2
hello3 + 0
time second = 31659
position and Best Price is True
86 37 30
time second = 31659
position and Best Price is True
87 37 31
diff_ask = 3
hello1 + 0
position and Best Price is True
88 37 32
diff_bid = 11


/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


hello3 + 0
time second = 31666
position and Best Price is True
89 38 32
diff_ask = 0
hello1 + 0
time second = 31666
position and Best Price is True
90 38 33
time second = 31686
position and Best Price is True
91 39 33
time second = 31686
position and Best Price is True
92 39 34
time second = 31695
position and Best Price is True
93 40 34
diff_ask = 3
hello1 + 0
position and Best Price is True
94 40 35
time second = 31749
position and Best Price is True
95 41 35
time second = 31759
position and Best Price is True
96 42 35
time second = 31759
position and Best Price is True
97 43 35
time second = 31765
position and Best Price is True
98 44 35
position and Best Price is True
99 44 36
diff_ask = 2
hello1 + 0
time second = 31765
position and Best Price is True
100 44 37
diff_ask = 0
hello1 + 0
time second = 31765
position and Best Price is True
101 44 38
time second = 31779
position and Best Price is True
102 45 38
diff_ask = -6
hello2 + 0
diff_ask = -6
hello2 + 1
diff_ask = -6
hello2 + 2
d

In [28]:
#data[(data.BidOrAsk == 'B')].TimeStamp.unique()[:]

In [45]:
#data[(data.BidOrAsk == 'A')].TimeStamp.unique()[0:]

In [27]:
import time
start = time.time()
time_ = data.TimeStamp.unique()
time_bid = data[(data.BidOrAsk == 'B')].TimeStamp.unique()[1:]
time_ask = data[(data.BidOrAsk == 'A')].TimeStamp.unique()[1:]
bid_index = 0
ask_index = 0
a = 0
b = 0
first = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
first = pd.DataFrame(first.values)
time = pd.DataFrame([['TimeStamp',time_[0][11:29],'','']])
order_book_concat = time.append(first).reset_index(drop=True)
order_book = order_book_concat
#print order_book
time_series = 1
for time in time_[1:]:
    #print 'time' + time
    #print 'time_series = %d'%(time_series)
    time_to_concat = pd.DataFrame([['TimeStamp',time_[time_series][11:29],'','']])
    #print time_to_concat
    #print time_bid[a]
    #print time_ask[b]
    #print order_book_bid_merge[a]
    #print order_book_ask_merge[b]
    if time == time_bid[a] and time == time_ask[b]:
        #print '1 = ' + time
        bid_index = bid_index + 1
        ask_index = ask_index + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
        b = b + 1
    elif time == time_bid[a] and time != time_ask[b]: 
        #print '2 = ' +time
        bid_index = bid_index + 1        
        
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
    elif time != time_bid[a] and time == time_ask[b]: 
        #print '3 = ' +time
        ask_index = ask_index + 1    
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        b = b + 1
    else:
        continue
    order_book = pd.concat([order_book,order_book_concat])    
    time_series = time_series + 1    
    
    #print order_book
end = time.time()            
print "Total time = %f"%(end - start) 


IndexError: index 43236 is out of bounds for axis 0 with size 43236

In [31]:
order_book = pd.DataFrame(order_book.values)
order_book.to_csv('CNF14_01_03_order_book_final.csv',sep = '\t',encoding = 'utf-8')

In [32]:
import time 
start  =  time.time()
for i in range(len(y),1000,1):
    #print '------------------------------------------------------------------------------'
    #print (i)
    #print data[['TimeStamp']][i:i+1]
    #print data[['Price','OrderNumber','QuantityDifference','BidOrAsk','BestPrice','OrderBookPosition']][i:i+1]
    #print 'data_ask_number = %d'%(data_ask_number)
    #print data.BidOrAsk[i] == 'A'
    if data.BidOrAsk[i] == 'A':
        #data_ask_number = data_ask_number + 1
        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            
            if data.TimeStamp[i] == x1[temp_ask]:
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1

                order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)

                if position_ == 0 and len(order_book_ask[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Ask & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass
                elif 0 < position_< (len(order_book_ask[temp_ask])-1):
                    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:#or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Ask & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'   
                        pass
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Ask & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                
                        pass
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True' 
                    pass
                
            elif data.TimeStamp[i] != x1[temp_ask]:
                ###a.append(x1[temp_ask])
                #print order_book_ask[temp_ask]['Price'][0:1].iloc[0]
                #print data_ask[data_ask_number:data_ask_number+1]['BestPrice'].iloc[0]
                #print data_ask[data_ask_number-2:data_ask_number-1]['BestPrice'].iloc[0]
                #print 'data_ask_number = %d'%(data_ask_number)
                #if data_ask_number == 0:
                    #print 'Best Price is True'
                #    pass
                #elif data_ask_number > 0 and order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data_ask[data_ask_number-2:data_ask_number-1]['BestPrice'].iloc[0]:
                    #print 'Best Price is True'
                #    pass
                #else:
                    #print 'Best Price is False'
                #    break
                #print temp_ask
                
                #if order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data[(data.BidOrAsk == 'A')][i:i]['BestPrice'].iloc[0]:
                #    print 'Best Price is True'
                #    continue
                #else:
                #    print 'Best Price is False'
                #   break
                    
                if temp_ask == 0:
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                    #print order_book_ask[temp_ask]
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    #print order_book_ask[temp_ask]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    #print 'position = %d'%(position_)
                    #print 'length-1 = %d'%(len(order_book_ask[temp_ask])-1)
                    #print order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]
                    #print order_book_ask[temp_ask][position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]
                    #print order_book_ask[temp_ask]['Price'][0:1].iloc[0]# != data['BestPrice'][i]
                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_ask[temp_ask])-1:   
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            pass
                        else:
                            print 'position and Best Price is True'
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        pass
                    #print order_book_ask[temp_ask]                        
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    order_book_ask.append(order_book_ask[temp_ask])
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                    if position_ == 0:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error3(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error3(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:    
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error3(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            pass
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        pass
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x1[temp_ask]: 
                #print temp_ask
                #print order_book_ask[temp_ask]
                #position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                
                #if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                #    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                #else:
                #    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_ask[temp_ask]

                if position_ == 0 and len(order_book_ask[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error4(Ask & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass       
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error4(Ask & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'  
                        pass
                elif position_ == len(order_book_ask[temp_ask])-1:       
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error4(Ask & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])   
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        pass                        
                    else:
                        print 'position and Best Price is True'                        
                        pass
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True'
                    pass
    
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]

            elif data.TimeStamp[i] != x1[temp_ask]:
                ###a.append(x1[temp_ask])
                #print temp_ask
                #print order_book_ask[temp_ask]
                #print order_book_ask[temp_ask]['Price'][0:1].iloc[0]
                #print data_ask[data_ask_number-2:data_ask_number-1]['BestPrice'].iloc[0]
                
                #if data_ask_number == 0:
                #    print 'Best Price is True'
                #    pass
                #elif data_ask_number > 0 and order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data_ask[data_ask_number-2:data_ask_number-1]['BestPrice'].iloc[0]:
                #    print 'Best Price is True'
                #    pass
                #else:
                #    print 'Best Price is False'
                #    break
                #if order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data['BestPrice'][i-1]:
                #    print 'Best Price is True'
                #    continue
                #else:
                #    print 'Best Price is False'
                #    break
                    
                order_book_ask.append(order_book_ask[temp_ask])
                #position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_ask = temp_ask + 1
                #print 'temp_ask2 = %d'%(temp_ask)
                #if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                #    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                #else:
                #    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_ask[temp_ask]
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error5(Ask & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error5(Ask & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                        break
                    else:
                        print 'position and Best Price is True'  
                        pass
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error5(Ask & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        pass                       
                    else:
                        print 'position and Best Price is True'                        
                        pass
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True'
                    pass      
                    
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                
                
                ##if data_ask_number == 0:
                ##    print 'Best Price is True'
                ##    pass
                ##elif data_ask_number > 0 and order_book_ask[temp_ask-1]['Price'][0:1].iloc[0] == data_ask[data_ask_number-2:data_ask_number-1]['BestPrice'].iloc[0]:
                ##    print 'Best Price is True'
                ##    pass
                ##else:
                ##    print 'Best Price is False'
                ##    break
   
                    
    elif data.BidOrAsk[i] == 'B':
        #print i
        #print int(data[['QuantityDifference']][i:i+1].values) > 0
        #data_bid_number = data_bid_number + 1
        #print data_bid_number
        
        if int(data[['QuantityDifference']][i:i+1].values) > 0: #and len(order_book_bid[0]) > 0:
            #print x2[temp_bid]
            #print data.TimeStamp[i] == x2[temp_bid]
            
            if data.TimeStamp[i] == x2[temp_bid]:
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Bid & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass     
                elif 0 < position_< len(order_book_bid[temp_bid])-1: 
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Bid & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'   
                        pass
                elif position_ == len(order_book_bid[temp_bid])-1 and len(order_book_bid[0]) > 1:      
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1(Bid & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        print 'position and Best Price is True'                
                        pass
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    pass  
            elif data.TimeStamp[i] != x2[temp_bid]:
                
                #print 'order book best price = %d'%(order_book_bid[temp_bid]['Price'][0:1].iloc[0])
                #print ''
                #print 'last data best price = %d'%(data_bid[data_bid_number-2:data_bid_number-1]['BestPrice'].iloc[0])
                #print data_bid_number
                
                #if data_bid_number == 0:
                    #print 'Best Price is True'
                #    pass
                #elif data_bid_number > 0 and order_book_bid[temp_bid]['Price'][0:1].iloc[0] == data_bid[data_bid_number-2:data_bid_number-1]['BestPrice'].iloc[0]:
                    #print 'Best Price is True'
                #    pass
                #else:
                    #print 'Best Price is False'
                #    break
                #data_bid_number = data_bid_number + 1   
                ###b.append(x2[temp_bid])
                #print 'temp bid = %d'%(temp_bid)
                #print order_book_bid[temp_bid]
                    
                if temp_bid == 0:
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                    #print order_book_bid[temp_bid]
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    #print order_book_bid[temp_bid]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    #print position_
                    #print len(order_book_bid[temp_bid]) - 1
                    
                    #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]
                    #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]
                    #print order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]
                    #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0]
                    #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0]
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    #print data['BestPrice'][i]
                    #print order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0]
                    if position_ == 0  and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass       
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1:    
                        if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])                                        
                            pass
                        else:
                            print 'position and Best Price is True'
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                            print 'position and Best Price is True'
                            pass
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    order_book_bid.append(order_book_bid[temp_bid])
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                    #print position_    
                    #print len(order_book_bid[temp_bid])-1      
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error3(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error3(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error3(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        pass
                        
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x2[temp_bid]: 
                #print (temp_bid)
                #print (order_book_bid[temp_bid])
                #position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                #print order_book_bid[temp_bid]
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error4(Bid & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        #print 'position and Best Price is True'                     
                        pass
                        
                elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error4(Bid & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                        break
                    else:
                        #print 'position and Best Price is True'  
                        pass
                elif position_ == len(order_book_bid[temp_bid])-1:
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                        print 'Some error4(Bid & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                        break
                    elif position_ == 0:
                        #print 'position and Best Price is True'
                        pass
                    else:
                        #print 'position and Best Price is True'                        
                        pass
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    #print 'position and Best Price is True'
                    pass
                    
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                              
                    
            elif data.TimeStamp[i] != x2[temp_bid]:

                #print 'order book best price = %d'%(order_book_bid[temp_bid]['Price'][0:1].iloc[0])
                #print ''
                #print 'last data best price = %d'%(data_bid[data_bid_number-2:data_bid_number-1]['BestPrice'].iloc[0])
                #print data_bid_number
                
                #if data_bid_number == 0:
                #    print 'Best Price is True'
                #    pass
                #elif data_bid_number > 0 and order_book_bid[temp_bid]['Price'][0:1].iloc[0] == data_bid[data_bid_number-2:data_bid_number-1]['BestPrice'].iloc[0]:
                #    print 'Best Price is True'
                #    pass
                #else:
                #    print 'Best Price is False'
                #    break
                
                #if order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data['BestPrice'][i-1]:
                #    print 'Best Price is True'
                #    continue
                #else:
                #    print 'Best Price is False'
                #    break                
                ###b.append(x2[temp_bid])
                #print temp_bid
                #print order_book_bid[temp_bid]
                order_book_bid.append(order_book_bid[temp_bid])
                #position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_bid = temp_bid + 1
                #print position_
                #print len(order_book_ask[temp_ask])-1
                #print 'temp_bid2 = %d'%(temp_bid)
                
                #if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                #    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                #else:
                #    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_bid[temp_bid]
                
                
                #print position_
                #print len(order_book_ask[temp_ask])-1
                #print order_book_bid[temp_bid]
                #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0]
                #print data['Price'][i:i+1].iloc[0]

                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error5(Bid & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        pass
    
                elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error5(Bid & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        pass
                elif position_ == len(order_book_bid[temp_bid])-1:
                        
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error5(Bid & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        pass
                    else:
                        print 'position and Best Price is True'                        
                        pass    
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    pass  
                    
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_bid[temp_bid]
                
                ##print 'order book best price = %d'%(order_book_bid[temp_bid]['Price'][0:1].iloc[0])
                ##print ''
                ##print 'last data best price = %d'%(data_bid[data_bid_number-2:data_bid_number-1]['BestPrice'].iloc[0])
                
                ##if data_bid_number == 0:
                ##    print 'Best Price is True'
                ##    pass
                ##elif data_bid_number > 0 and order_book_bid[temp_bid-1]['Price'][0:1].iloc[0] == data_bid[data_bid_number-2:data_bid_number-1]['BestPrice'].iloc[0]:
                ##    print 'Best Price is True'
                ##    pass
                ##else:
                ##    print 'Best Price is False'
                ##    break       
                                    
end = time.time()            
print "Total time = %f"%(end - start)

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position and Best Price is True
position

In [30]:
import time
start = time.time()


time_ = data.TimeStamp.unique()
time_bid = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
time_ask = data[(data.BidOrAsk == 'A')].TimeStamp.unique()

bid_index = 0
ask_index = 0
a = 0
b = 0
first = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
first = pd.DataFrame(first.values)
time = pd.DataFrame([['TimeStamp',time_[0][11:29],'','']])
order_book_concat = time.append(first).reset_index(drop=True)
order_book = order_book_concat
#print order_book
a = 1
b = 1
time_series = 1
for time in time_[1:]:
    #print 'time' + time
    #print 'time_series = %d'%(time_series)
    time_to_concat = pd.DataFrame([['TimeStamp',time_[time_series][11:29],'','']])
    #print time_to_concat
    #print time_bid[a-1]
    #print time_ask[b-1]    
    #print time_bid[a]
    #print time_ask[b]
    #print order_book_bid_merge[a]
    #print order_book_ask_merge[b]
    if time == time_bid[a] and time == time_ask[b]:
        #print '1 = ' + time
        #a = a + 1
        #b = b + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[a].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[b].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
        b = b + 1
    elif time == time_bid[a] and time != time_ask[b]: 
        #print '2 = ' +time
        ##a = a + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[a].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[b-1].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
        
    elif time != time_bid[a] and time == time_ask[b]: 
        #print '3 = ' +time
        ##b = b + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[a-1].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[b].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        b = b + 1
    else:
        continue
    order_book = pd.concat([order_book,order_book_concat])    
    time_series = time_series + 1    
    #print order_book
end = time.time()            
print "Total time = %f"%(end - start) 


IndexError: index 45578 is out of bounds for axis 0 with size 45578

In [7]:
order_book = pd.DataFrame(order_book.values)
order_book.to_csv('CNF14_01_13_order_book_final.csv',sep = '\t',encoding = 'utf-8')